In [0]:
!pip install pandas 
!pip install numpy

In [0]:
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', 500)

In [0]:
# function to normalize along the mean
def norm(data_frame, col):
    a = data_frame[col]
    mean = a.mean()
    sigma = a.std()
    
    a = (a - mean)/sigma
    return a

In [0]:
# checking if the year is a leap year
def is_leap(year):
    if year%4:
        return False
    elif year%100:
        return False
    elif year%400:
        return False
    else:
        return True

In [0]:
# getting days 
def give_allowed_days(month, year):
    allowed_days = 0
    if month in [1, 3, 5, 7, 8, 10, 12]:
        allowed_days = 31
    elif month in [4, 6, 9, 11]:
        allowed_days = 30
    else:
        if is_leap(year):
            allowed_days = 29
        else:
            allowed_days = 28
    return allowed_days

In [0]:
def day_corrector(false_date, month, year):
    '''false_date, month, year are np.array(s)'''
    for i in range(len(false_date)):
        # getting max days in a month
        allowed_days = give_allowed_days(month[i], year[i])

        if false_date[i] > allowed_days:
            false_date[i] = allowed_days
    return false_date, month, year

In [0]:
def make_year_claim(month_acc, month_claim, week_claim, week_acc, day_claim, day_acc, year_acc):
    ret_year = []
    for i in range(len(year_acc)):
        if month_claim[i] < month_acc[i]:
            ret_year.append(year_acc[i] + 1)
        elif month_claim[i] == month_acc[i]:
            if week_claim[i] < week_acc[i]:
                ret_year.append(year_acc[i] + 1)
            elif week_claim[i] == week_acc[i]:
                if day_claim[i] < day_acc[i]:
                    ret_year.append(year_acc[i] + 1)
                else:
                    ret_year.append(year_acc[i])
            else:
                ret_year.append(year_acc[i])
        else:
            ret_year.append(year_acc[i])
    return ret_year

In [0]:
# uncomment these lines for preprocessing submission file
# in submission file first removed all editing and extra sheets
df = pd.read_csv('./sample_output.csv')

# uncomment the following to preprocess training data
# df = pd.read_csv('./exl/Dataset.csv')

In [186]:
df.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,ClaimSize,FraudFound_P
0,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,27,Third Party,Sport - Collision,Sport,more than 69000,5,3,400,1,more than 30,more than 30,none,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision,56711.16,NaN
1,Oct,4,Friday,Honda,Urban,Wednesday,Nov,1,Male,Single,20,Third Party,Sport - Collision,Sport,more than 69000,6,12,400,3,more than 30,more than 30,none,5 years,21 to 25,No,No,External,3 to 5,no change,1 vehicle,1994,Collision,60746.44,NaN
2,Nov,1,Friday,Honda,Urban,Tuesday,Mar,4,Male,Single,0,Policy Holder,Sport - Collision,Sport,more than 69000,8,1,400,4,more than 30,more than 30,1,new,16 to 17,No,No,External,none,no change,1 vehicle,1994,Collision,74173.64,NaN
3,Apr,3,Tuesday,Ford,Urban,Wednesday,Apr,3,Male,Married,42,Policy Holder,Utility - All Perils,Utility,more than 69000,10,7,400,1,more than 30,more than 30,2 to 4,more than 7,36 to 40,No,No,External,3 to 5,no change,1 vehicle,1994,All Perils,2482.01,NaN
4,Apr,4,Monday,Honda,Urban,Tuesday,May,1,Male,Married,30,Third Party,Sedan - Liability,Sport,more than 69000,20,2,400,2,more than 30,more than 30,2 to 4,6 years,31 to 35,No,No,External,more than 5,no change,1 vehicle,1994,Liability,88101.03,NaN


In [0]:
# uncomment this for pre-processing of submission data.
# comment this when processing training data
df.drop(columns='FraudFound_P', axis=1, inplace=True)

In [0]:
# only took those examples that have a valid date
df = df[df['DayOfWeekClaimed'] != '0']

In [0]:
# dropped irrelevant feature
df = df.drop(columns=['PolicyNumber'], axis=1)

month_mapper = {'Dec':12, 'Jan':1, 'Oct':10, 'Jun':6, 'Feb':2, 'Mar':3, 'Aug':8, 'Apr':4, 'Jul':7, 'May':5, 'Sep':9, 'Nov':11, '0':0}
week_mapper = {'Wednesday':3, 'Friday':5, 'Saturday':6, 'Sunday':7, 'Monday':1, 'Tuesday':2, 'Thursday':4, '0':0}


df['Month'] = df['Month'].map(month_mapper)
df['DayOfWeek'] = df['DayOfWeek'].map(week_mapper)
df['MonthClaimed'] = df['MonthClaimed'].map(month_mapper)
df['DayOfWeekClaimed'] = df['DayOfWeekClaimed'].map(week_mapper)

# df['MonthClaimed'] = df['MonthClaimed'].astype('int')
# df['DayOfWeekClaimed'] = df['DayOfWeekClaimed'].astype('int')

df = df.dropna()

In [0]:
# getting the date of month
df['Accident_date'] = ((df['WeekOfMonth'] - 1) * 7) + df['DayOfWeek']
df['claim_date'] = ((df['WeekOfMonthClaimed'] - 1) * 7) + df['DayOfWeekClaimed']

In [0]:
# correcting date if date exceeds allowed number of days that is 31, 30, 28 or 29 depending on situation
df['Accident_date'], df['Month'], df['Year'] = day_corrector(np.asarray(df['Accident_date']), np.asarray(df['Month']), np.asarray(df['Year']))

In [0]:
# assuming if claim is befor accident then application is filed 1 year after the accident
df['year_claim'] = make_year_claim(np.asarray(df['Month']),
                                   np.asarray(df['MonthClaimed']),
                                   np.asarray(df['WeekOfMonthClaimed']),
                                   np.asarray(df['WeekOfMonth']),
                                   np.asarray(df['DayOfWeekClaimed']),
                                   np.asarray(df['DayOfWeek']),
                                   np.asarray(df['Year']),
                                  )

In [0]:
df['claim_date'], df['MonthClaimed'], df['year_claim'] = day_corrector(np.asarray(df['claim_date']), np.asarray(df['MonthClaimed']), np.asarray(df['year_claim']))

In [0]:
# making a datetime of 'accident date' and 'claim date'
df['Accident_datetime'] = pd.to_datetime((df.Year*10000+df.Month*100+df.Accident_date).apply(str), format='%Y%m%d')
df['claim_datetime'] = pd.to_datetime((df.year_claim*10000+df.MonthClaimed*100+df.claim_date).apply(str), format='%Y%m%d')

In [0]:
# label encoding for varius columns
Sex = {'Female':1, 'Male':0}
MaritalStatus = {'Single':1, 'Married':2, 'Widow':3, 'Divorced':4}
Make = {'Honda':1, 'Toyota':2, 'Mazda':3, 'Ford':4, 'Chevrolet':5, 'Pontiac':6, 'Dodge':7, 'Accura':8, 'Mercury':9, 'Jaguar':10, 'Nisson':11, 'VW':12, 'Saab':13, 'Saturn':14, 'Porche':15, 'BMW':16, 'Mecedes':17, 'Ferrari':18, 'Lexus':19}
AccidentArea = {'Urban':1, 'Rural':2}
Fault ={'Policy Holder':1, 'Third Party':2}
PolicyType = {'Sport - Liability':1, 'Sport - Collision':2, 'Sedan - Liability':3, 'Sedan - All Perils':4, 'Sedan - Collision':5, 'Utility - Collision':6, 'Utility - Liability':7, 'Utility - All Perils':8, 'Sport - All Perils':9}
VehicleCategory = {'Sport':1, 'Sedan':2, 'Utility':3}
VehiclePrice = {'more than 69000':6, '20000 to 29000':2, '30000 to 39000':3, 'less than 20000':1, '40000 to 59000':4, '60000 to 69000':5}
Days_Policy_Accident = {'more than 30':4, '15 to 30':3, 'none':0, '1 to 7':1, '8 to 15':2}
Days_Policy_Claim = {'more than 30':4, '15 to 30':3, 'none':0, '1 to 7':1, '8 to 15':2}
PastNumberOfClaims = {'none':0, '1':1, '2 to 4':3, 'more than 4':4}
AgeOfVehicle = {'3 years':3, '6 years':6, '7 years':7, 'more than 7':8, 'new':8, '5 years':5, '4 years':4, '2 years':2, '1 year':1, '1 years':1}
AgeOfPolicyHolder = {'26 to 30':4, '31 to 35':5, '41 to 50':7, '51 to 65':8, '36 to 40':6, 'over 65':9, '16 to 17':1, '18 to 20':2, '21 to 25':3}
PoliceReportFiled = {'No':0, 'Yes':1}
WitnessPresent = {'No':0, 'Yes':1}
AgentType = {'External':0, 'Internal':1}
NumberOfSuppliments = {'none':0, 'more than 5':3, '1 to 2':1, '3 to 5':2}
AddressChange_Claim = {'1 year':2, 'no change':0, '4 to 8 years':4, '2 to 3 years':3, 'under 6 months':1}
NumberOfCars = {'3 to 4':3, '1 vehicle':1, '2 vehicles':2, '5 to 8':4, 'more than 8':5}
BasePolicy = {'Liability':1, 'Collision':2, 'All Perils':3}

In [0]:
df['Fault'] = df['Fault'].map(Fault)
df['Make'] = df['Make'].map(Make)
df['AccidentArea'] = df['AccidentArea'].map(AccidentArea)
df['Sex'] = df['Sex'].map(Sex)
df['MaritalStatus'] = df['MaritalStatus'].map(MaritalStatus)
df['PolicyType'] = df['PolicyType'].map(PolicyType)
df['VehicleCategory'] = df['VehicleCategory'].map(VehicleCategory)
df['VehiclePrice'] = df['VehiclePrice'].map(VehiclePrice)
df['Days_Policy_Accident'] = df['Days_Policy_Accident'].map(Days_Policy_Accident)
df['Days_Policy_Claim'] = df['Days_Policy_Claim'].map(Days_Policy_Claim)
df['PastNumberOfClaims'] = df['PastNumberOfClaims'].map(PastNumberOfClaims)
df['AgeOfVehicle'] = df['AgeOfVehicle'].map(AgeOfVehicle)
df['AgeOfPolicyHolder'] = df['AgeOfPolicyHolder'].map(AgeOfPolicyHolder)
df['PoliceReportFiled'] = df['PoliceReportFiled'].map(PoliceReportFiled)
df['WitnessPresent'] = df['WitnessPresent'].map(WitnessPresent)
df['AgentType'] = df['AgentType'].map(AgentType)
df['NumberOfSuppliments'] = df['NumberOfSuppliments'].map(NumberOfSuppliments)
df['AddressChange_Claim'] = df['AddressChange_Claim'].map(AddressChange_Claim)
df['NumberOfCars'] = df['NumberOfCars'].map(NumberOfCars)
df['BasePolicy'] = df['BasePolicy'].map(BasePolicy)

In [0]:
df.drop(columns=['PoliceReportFiled', 'WitnessPresent', 'Days_Policy_Claim'], axis=1, inplace=True)

In [0]:
df['days_btw_claim'] = df['claim_datetime'] - df['Accident_datetime']

In [0]:
# converting time delta into float
df['days_btw_claim'] = df['days_btw_claim'].apply(lambda x: x.total_seconds()/(60*60*24))

In [0]:
# converting datetime into timedelta and extracting no of days
df['Accident_datetime'] = df['Accident_datetime'].apply(lambda x: (datetime.datetime.today() - x).days)
df['claim_datetime'] = df['claim_datetime'].apply(lambda x: (datetime.datetime.today() - x).days)

In [0]:
# making a feature if the applicant is male, married and between age 30-50
df['middleAged_marriedMan'] = list(np.logical_and(np.logical_or(df.AgeOfPolicyHolder == 5, np.logical_or(df.AgeOfPolicyHolder == 6, df.AgeOfPolicyHolder == 7)), np.logical_and(df.Sex == 0, df.MaritalStatus == 2))) 
df['middleAged_marriedMan'] = df['middleAged_marriedMan'].map({False:0, True:1})

In [0]:
# normalizing columns
cols = ['Deductible', 'ClaimSize', 'Age', 'Year', 'year_claim', 'Accident_datetime', 'claim_datetime', 'days_btw_claim']
for col in cols:
    df[col] = norm(df, col)

In [0]:
# uncomment when processing submission data
# df.to_csv('./subPreorocessedData.csv', index=False)

# uncomment when processing training dateset
# df.to_csv('./datePracessedData.csv', index=False)